# Africa Conflict Data Report From ACLED

# Kuek Philip Atem Marol   SDS6/46832/2024

# Download From Github or Google drive

In [2]:
import pandas as pd
import plotly.express as px
import folium
from folium.plugins import MarkerCluster
import dash
from dash import dcc, html, Input, Output, dash_table
import os
import markdown
from fpdf import FPDF

# Load dataset
file_path = "/Users/ghost/Documents/69/Kuek/Gisma/Projects/UON/Africa_1997-2024_Dec13.csv"
df = pd.read_csv(file_path)
df['event_date'] = pd.to_datetime(df['event_date'])

# Function to generate markdown report
def generate_markdown_report(filtered_df):
    report = """# Africa Conflict Data Report From ACLED

## Summary Statistics

- Total Events: {}
- Total Fatalities: {}

## Event Distribution by Type

""".format(len(filtered_df), filtered_df["fatalities"].sum())
    
    bar_df = filtered_df["event_type"].value_counts().reset_index()
    bar_df.columns = ["event_type", "count"]
    for _, row in bar_df.iterrows():
        report += "- {}: {} events\n".format(row["event_type"], row["count"])
    
    return report

# Function to generate PDF report
def generate_pdf_report(filtered_df, filename="Africa_Conflict_Report.pdf"):
    pdf = FPDF()
    pdf.set_auto_page_break(auto=True, margin=15)
    pdf.add_page()
    pdf.set_font("Arial", "B", 16)
    pdf.cell(200, 10, "Africa Conflict Data Report From ACLED", ln=True, align="C")
    pdf.ln(10)
    pdf.set_font("Arial", size=12)
    pdf.cell(200, 10, "Summary Statistics", ln=True, align="L")
    pdf.cell(200, 10, f"Total Events: {len(filtered_df)}", ln=True, align="L")
    pdf.cell(200, 10, f"Total Fatalities: {filtered_df['fatalities'].sum()}", ln=True, align="L")
    pdf.ln(10)
    pdf.cell(200, 10, "Event Distribution by Type", ln=True, align="L")
    
    bar_df = filtered_df["event_type"].value_counts().reset_index()
    bar_df.columns = ["event_type", "count"]
    for _, row in bar_df.iterrows():
        pdf.cell(200, 10, f"- {row['event_type']}: {row['count']} events", ln=True, align="L")
    
    pdf.output(filename)
    return filename

# Initialize Dash app
app = dash.Dash(__name__)

# Layout
app.layout = html.Div([
    html.H1("Africa Conflict Data Dashboard From ACLED"),
    dcc.Dropdown(
        id="country",
        options=[{"label": c, "value": c} for c in sorted(df["country"].unique())],
        value=None,
        placeholder="Select a country",
    ),
    dcc.DatePickerRange(
        id="date_range",
        start_date=df["event_date"].min(),
        end_date=df["event_date"].max(),
    ),
    dcc.Dropdown(
        id="event_type",
        options=[{"label": e, "value": e} for e in sorted(df["event_type"].unique())],
        value=None,
        placeholder="Select event type",
    ),
    dcc.Slider(
        id="fatalities",
        min=0,
        max=df["fatalities"].max(),
        value=0,
        marks={i: str(i) for i in range(0, df["fatalities"].max(), max(1, df["fatalities"].max() // 5))},
    ),
    dash_table.DataTable(id="table"),
    dcc.Graph(id="time_series_plot"),
    dcc.Graph(id="bar_chart"),
    html.Div(id="map"),
    html.Button("Download Markdown Report", id="download_md_button"),
    html.Button("Download PDF Report", id="download_pdf_button"),
    dcc.Download(id="download_md_report"),
    dcc.Download(id="download_pdf_report")
])

# Callbacks
@app.callback(
    [Output("table", "data"),
     Output("time_series_plot", "figure"),
     Output("bar_chart", "figure"),
     Output("map", "children")],
    [Input("country", "value"),
     Input("date_range", "start_date"),
     Input("date_range", "end_date"),
     Input("event_type", "value"),
     Input("fatalities", "value")]
)
def update_dashboard(country, start_date, end_date, event_type, fatalities):
    filtered_df = df.copy()
    if country:
        filtered_df = filtered_df[filtered_df["country"] == country]
    if event_type:
        filtered_df = filtered_df[filtered_df["event_type"] == event_type]
    filtered_df = filtered_df[(filtered_df["event_date"] >= start_date) & (filtered_df["event_date"] <= end_date)]
    filtered_df = filtered_df[filtered_df["fatalities"] >= fatalities]
    
    ts_df = filtered_df.groupby("event_date").size().reset_index(name="count")
    time_series_fig = px.line(ts_df, x="event_date", y="count", title="Event Trend Over Time")
    
    bar_df = filtered_df["event_type"].value_counts().reset_index()
    bar_df.columns = ["event_type", "count"]
    bar_chart_fig = px.bar(bar_df, x="event_type", y="count", title="Event Type Distribution")
    
    m = folium.Map(location=[0, 25], zoom_start=3)
    marker_cluster = MarkerCluster().add_to(m)
    for _, row in filtered_df.iterrows():
        folium.Marker(
            location=[row["latitude"], row["longitude"]],
            popup=f"{row['event_type']} in {row['location']} on {row['event_date']}",
        ).add_to(marker_cluster)
    map_html = m._repr_html_()
    
    return filtered_df.to_dict("records"), time_series_fig, bar_chart_fig, html.Iframe(srcDoc=map_html, width="100%", height="500px")

@app.callback(
    Output("download_md_report", "data"),
    Input("download_md_button", "n_clicks"),
    prevent_initial_call=True
)
def download_md_report(n_clicks):
    report_content = generate_markdown_report(df)
    return dict(content=report_content, filename="Africa_Conflict_Report.md")

@app.callback(
    Output("download_pdf_report", "data"),
    Input("download_pdf_button", "n_clicks"),
    prevent_initial_call=True
)
def download_pdf_report(n_clicks):
    filename = generate_pdf_report(df)
    return dict(content=open(filename, "rb").read(), filename=filename, type="application/pdf")

# Run the app
if __name__ == "__main__":
    app.run_server(debug=True)
